In [9]:
import numpy as np
from scipy import linalg

sx = 1/2 * np.mat([[0, 1],[ 1, 0]], dtype=complex)
sy = 1/2 * np.mat([[0, -1j],[1j, 0]], dtype=complex)
sz = 1/2 * np.mat([[1, 0],[0, -1]], dtype=complex)

def hamiltonian(j):
    J = 4
    H = (j) * J * sz + sx
    return H


T = 2*np.pi       
N = 40
dt = T/N

I = 500
fidelity = np.zeros(I+1)

observable = np.mat(np.zeros(shape=(2,2), dtype=complex))
observable[-1, -1] = 1

psi = np.mat(np.zeros(shape=(2, N+1), dtype=complex))
psi[0,0] = 1   
pseudo = np.mat(np.zeros(shape=(2, N+1), dtype=complex))    # 


seq = np.random.rand(N)
seq_f = seq

for i in range(N):
    psi[:,i+1] = linalg.expm(-(1j) * hamiltonian(seq[i]) * dt).dot(psi[:,i])
fidelity[0]=(np.absolute(psi[-1,-1]))**2
pseudo[:,-1] = observable.dot(psi[:,-1])
dj = 0.01    


for i in range(I):
    for j in reversed(range(N)):
        pseudo[:,j] = linalg.expm((1j) * hamiltonian(seq[j]) * dt).dot(pseudo[:,j+1])
    for k in range(N):
        dH = (hamiltonian(seq[k]+dj) - hamiltonian(seq[k]-dj)) / (2*dj)
        seq_f[k] = seq[k] + (pseudo[:,k].conj().T.dot(dH.dot(psi[:,k]))).imag[0,0]
        psi[:,k+1] = linalg.expm(-(1j) * hamiltonian(seq_f[k]) * dt).dot(psi[:,k])
        seq = seq_f
    fidelity[i+1] += (np.absolute(psi[-1,-1]))**2
    pseudo[:,-1] = observable.dot(psi[:,-1])


print('final_fidelity=',fidelity[-1])


final_fidelity= 1.000000000000003


In [5]:
import numpy as np
from scipy.linalg import expm

def cost(seq):

    N=len(seq) 

    dt=2*np.pi/N  

    sx=1/2 * np.mat([[0,1],\
                 [1,0]], dtype=complex)
    sz=1/2 * np.mat([[1,0],\
                 [0,-1]], dtype=complex)

    U = np.matrix(np.identity(2, dtype=complex)) #initial Evolution operator

    J=4                                   # control field strength

    for ii in seq:
        H =ii * J * sz + 1*sx # Hamiltonian
        U = expm(-1j * H * dt) * U  # Evolution operator

    p0=np.mat([[1],[0]], dtype=complex) #initial state
    pt=U * p0              #final state

    target = np.mat([[0], [1]], dtype=complex)                             # south pole

    err = 1-(np.abs(pt.H * target)**2).item(0).real            #infidelity (to make it as small as possible)

    return err



delta=0.01
cost_hist = []

def gradient_descent(x, dim, learning_rate, num_iterations):
    for i in range(num_iterations):
        v=np.random.rand(dim) 
        xp=x+v*delta
        xm=x-v*delta
        error_derivative = (cost(xp) - cost(xm))/(2*delta)
        x = x - (learning_rate) * error_derivative*v
        cost_hist.append(cost(xp))
    return cost(x)


N        = 20
seq      = np.random.rand(N)
ep_max   = 500
fidelity = 1-gradient_descent(seq, N, 0.01, ep_max)

print('Final_fidelity=',fidelity)    



Final_fidelity= 0.33890561435297184


In [3]:
import numpy as np
from scipy.linalg import expm

def cost(seq):

    N=len(seq) 

    dt=2*np.pi/N  

    sx=1/2 * np.mat([[0,1],\
                 [1,0]], dtype=complex)
    sz=1/2 * np.mat([[1,0],\
                 [0,-1]], dtype=complex)

    U = np.matrix(np.identity(2, dtype=complex)) #initial Evolution operator

    J=4                                   # control field strength

    for ii in seq:
        H =ii * J * sz + 1*sx # Hamiltonian
        U = expm(-1j * H * dt) * U  # Evolution operator

    p0=np.mat([[1],[0]], dtype=complex) #initial state
    pt=U * p0              #final state

    target = np.mat([[0], [1]], dtype=complex)                             # south pole

    err = 1-(np.abs(pt.H * target)**2).item(0).real            #infidelity (to make it as small as possible)

    return err



delta=0.01
cost_hist = []

def gradient_descent(x, dim, learning_rate, num_iterations):
    for i in range(num_iterations):
        v=np.random.rand(dim) 
        xp=x+v*delta
        xm=x-v*delta
        error_derivative = (cost(xp) - cost(xm))/(2*delta)
        x = x - (learning_rate) * error_derivative*v
        cost_hist.append(cost(xp))
    return cost(x)


N        = 100
seq      = np.random.rand(N)
ep_max   = 500
fidelity = 1-gradient_descent(seq, N, 0.01, ep_max)

print('Final_fidelity=',fidelity)    



Final_fidelity= 0.15169701648516487
